In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from env import get_connection
import os

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (env.py, line 51)

In [2]:
# create helper function to get the necessary connection url.
def get_db_connection(database):
    return get_connection(database)

# connect to sql zillow database
url = "zillow"

# use this query to get data    
sql_query = "SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips FROM properties_2017 WHERE propertylandusetypeid = 261"

# assign data to data frame
df = pd.read_sql(sql_query, get_connection(url))

NameError: name 'get_connection' is not defined